In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !nvidia-smi

In [3]:
# import os
# os.chdir('/content/drive/MyDrive/Twitter')

In [4]:
# !apt-get install openjdk-8-jdk-headless
# !wget https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
# !tar xf spark-3.4.1-bin-hadoop3.tgz

In [5]:
# !pip install pyspark

In [6]:
# !pip install findspark

In [7]:
# import findspark
# findspark.init()
import pyspark as ps
import warnings
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
# import pandas as pd
# import pyspark.pandas as ps
import numpy as np
import time

from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, Word2Vec
from nltk.stem.snowball import SnowballStemmer

from pyspark.ml.classification import LogisticRegression

from pyspark.ml.evaluation import BinaryClassificationEvaluator

**1. Đọc dữ liệu vào**

In [8]:
spark = SparkSession.builder\
        .master("local")\
        .appName('Sentiment Analysis')\
        .getOrCreate()

schema = StructType([\
                    StructField("target", IntegerType(), True),\
                    StructField("id", IntegerType(), True),\
                    StructField("date", DateType(), True),\
                    StructField("flag", StringType(), True),\
                    StructField("user", StringType(), True),\
                    StructField("text", StringType(), True)])

df = spark.read.csv('training.1600000.processed.noemoticon.csv', sep=',', header=False, schema=schema)
df.printSchema()

23/08/02 23:02:42 WARN Utils: Your hostname, MacBook-Pro-cua-awlvn.local resolves to a loopback address: 127.0.0.1; using 192.168.1.9 instead (on interface en0)
23/08/02 23:02:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/02 23:02:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- target: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- flag: string (nullable = true)
 |-- user: string (nullable = true)
 |-- text: string (nullable = true)



In [9]:
df.show(5)

+------+----------+----+--------+---------------+--------------------+
|target|        id|date|    flag|           user|                text|
+------+----------+----+--------+---------------+--------------------+
|     0|1467810369|null|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|     0|1467810672|null|NO_QUERY|  scotthamilton|is upset that he ...|
|     0|1467810917|null|NO_QUERY|       mattycus|@Kenichan I dived...|
|     0|1467811184|null|NO_QUERY|        ElleCTF|my whole body fee...|
|     0|1467811193|null|NO_QUERY|         Karoli|@nationwideclass ...|
+------+----------+----+--------+---------------+--------------------+
only showing top 5 rows



In [10]:
df.head()

Row(target=0, id=1467810369, date=None, flag='NO_QUERY', user='_TheSpecialOne_', text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D")

**2. Tiền xử lý**

In [11]:
df.groupBy("target").count().show()

+------+------+
|target| count|
+------+------+
|     4|800000|
|     0|800000|
+------+------+



=> như vậy dữ liệu tích cực (target=4) và tiêu cực (target=0) là cân bằng, không có trường hợp nào target null cần xử lý

Chỉ quan tâm đến các trường dùng để đánh giá (id, target, text)

In [12]:
df1 = df.select("id", "target", "text")

In [38]:
df_clean = df1.select('*', (lower(regexp_replace('text', "[^a-zA-Z\\s]", "")).alias('newtext')))
df_clean.head()

Row(id=1467810369, target=0, text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", newtext='switchfoot httptwitpiccomyzl  awww thats a bummer  you shoulda got david carr of third day to do it d')

In [39]:
tokenizer = Tokenizer(inputCol='newtext', outputCol='tokens')
df_tokens = tokenizer.transform(df_clean)
df_tokens.head()

Row(id=1467810369, target=0, text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", newtext='switchfoot httptwitpiccomyzl  awww thats a bummer  you shoulda got david carr of third day to do it d', tokens=['switchfoot', 'httptwitpiccomyzl', '', 'awww', 'thats', 'a', 'bummer', '', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', 'd'])

In [44]:
filter_empty_udf = udf(lambda tokens: [token for token in tokens if len(token) > 0], ArrayType(StringType()))

In [45]:
df_no_empty = df_tokens.withColumn("no_empty", filter_empty_udf("tokens"))
df_no_empty.head()

Row(id=1467810369, target=0, text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", newtext='switchfoot httptwitpiccomyzl  awww thats a bummer  you shoulda got david carr of third day to do it d', tokens=['switchfoot', 'httptwitpiccomyzl', '', 'awww', 'thats', 'a', 'bummer', '', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', 'd'], no_empty=['switchfoot', 'httptwitpiccomyzl', 'awww', 'thats', 'a', 'bummer', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', 'd'])

In [47]:
remover = StopWordsRemover(inputCol="no_empty", outputCol="words_clean")
df_words_no_stopw = remover.transform(df_no_empty)
df_words_no_stopw.head()

Row(id=1467810369, target=0, text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", newtext='switchfoot httptwitpiccomyzl  awww thats a bummer  you shoulda got david carr of third day to do it d', tokens=['switchfoot', 'httptwitpiccomyzl', '', 'awww', 'thats', 'a', 'bummer', '', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', 'd'], no_empty=['switchfoot', 'httptwitpiccomyzl', 'awww', 'thats', 'a', 'bummer', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', 'd'], words_clean=['switchfoot', 'httptwitpiccomyzl', 'awww', 'thats', 'bummer', 'shoulda', 'got', 'david', 'carr', 'third', 'day', 'd'])

Đưa các token về dạng từ gốc

In [48]:
locale = spark._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))

stemmer = SnowballStemmer(language='english')
stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
df_stemmed = df_words_no_stopw.withColumn("words_stemmed", stemmer_udf("words_clean"))
df_stemmed.head()

Row(id=1467810369, target=0, text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", newtext='switchfoot httptwitpiccomyzl  awww thats a bummer  you shoulda got david carr of third day to do it d', tokens=['switchfoot', 'httptwitpiccomyzl', '', 'awww', 'thats', 'a', 'bummer', '', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', 'd'], no_empty=['switchfoot', 'httptwitpiccomyzl', 'awww', 'thats', 'a', 'bummer', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', 'd'], words_clean=['switchfoot', 'httptwitpiccomyzl', 'awww', 'thats', 'bummer', 'shoulda', 'got', 'david', 'carr', 'third', 'day', 'd'], words_stemmed=['switchfoot', 'httptwitpiccomyzl', 'awww', 'that', 'bummer', 'shoulda', 'got', 'david', 'carr', 'third', 'day', 'd'])

vocab size

Đưa cột "target" về dạng index

In [49]:
label_stringIdx = StringIndexer(inputCol = "target", outputCol = "label")
label_df = label_stringIdx.fit(df_stemmed).transform(df_stemmed)
label_df.show(5)

+----------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|        id|target|                text|             newtext|              tokens|            no_empty|         words_clean|       words_stemmed|label|
+----------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|1467810369|     0|@switchfoot http:...|switchfoot httptw...|[switchfoot, http...|[switchfoot, http...|[switchfoot, http...|[switchfoot, http...|  0.0|
|1467810672|     0|is upset that he ...|is upset that he ...|[is, upset, that,...|[is, upset, that,...|[upset, cant, upd...|[upset, cant, upd...|  0.0|
|1467810917|     0|@Kenichan I dived...|kenichan i dived ...|[kenichan, i, div...|[kenichan, i, div...|[kenichan, dived,...|[kenichan, dive, ...|  0.0|
|1467811184|     0|my whole body fee...|my whole body fee...|[my, whole, body,...|[my, w

23/08/02 23:41:10 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 63 (TID 80): Attempting to kill Python Worker


In [71]:
label_df.sample(False, 0.1, seed=0).limit(1)

DataFrame[id: int, target: int, text: string, newtext: string, tokens: array<string>, no_empty: array<string>, words_clean: array<string>, words_stemmed: array<string>, label: double]

In [50]:
label_df.head()

Row(id=1467810369, target=0, text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", newtext='switchfoot httptwitpiccomyzl  awww thats a bummer  you shoulda got david carr of third day to do it d', tokens=['switchfoot', 'httptwitpiccomyzl', '', 'awww', 'thats', 'a', 'bummer', '', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', 'd'], no_empty=['switchfoot', 'httptwitpiccomyzl', 'awww', 'thats', 'a', 'bummer', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', 'd'], words_clean=['switchfoot', 'httptwitpiccomyzl', 'awww', 'thats', 'bummer', 'shoulda', 'got', 'david', 'carr', 'third', 'day', 'd'], words_stemmed=['switchfoot', 'httptwitpiccomyzl', 'awww', 'that', 'bummer', 'shoulda', 'got', 'david', 'carr', 'third', 'day', 'd'], label=0.0)

**3. Classification**

3.1. CountVectorizer + IDF + Logistic Regression

In [51]:
# %%time
# vectorizer = CountVectorizer(minDF=2, inputCol="words_stemmed", outputCol="cntVec")
# cntvec_df = vectorizer.fit(label_df).transform(label_df)
# cntvec_df.show(5)

In [52]:
# %%time
# idf = IDF(inputCol="cntVec", outputCol="idfVec", minDocFreq=5)
# idf_df = idf.fit(cntvec_df).transform(cntvec_df)
# idf_df.show(5)

In [53]:
# [train_idf_df, test_idf_df, val] = idf_df.randomSplit([0.1, 0.1, 0.8], seed = 123)

In [54]:
# %%time
# t1 = time.time()

# lr = LogisticRegression(featuresCol="idfVec")
# lrModel = lr.fit(train_idf_df)
# predictions = lrModel.transform(test_idf_df)

# t2 = time.time()
# print('Time process Logistic Regression: {0:.2f} seconds'.format(t2-t1) )

In [55]:
# evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
# evaluator.evaluate(predictions)

In [56]:
# accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(test_idf_df.count())
# accuracy

3.2. Word2Vec + Logistic Regression

In [61]:
%%time
word2vec = Word2Vec(vectorSize=100, minCount=5, inputCol = "words_stemmed", outputCol = "w2v", seed = 123)
w2v_df = word2vec.fit(label_df).transform(label_df)

CPU times: user 46.3 ms, sys: 38.2 ms, total: 84.5 ms
Wall time: 8min 11s


In [62]:
w2v_df.head()

23/08/02 23:54:02 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 82 (TID 108): Attempting to kill Python Worker


Row(id=1467810369, target=0, text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", newtext='switchfoot httptwitpiccomyzl  awww thats a bummer  you shoulda got david carr of third day to do it d', tokens=['switchfoot', 'httptwitpiccomyzl', '', 'awww', 'thats', 'a', 'bummer', '', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', 'd'], no_empty=['switchfoot', 'httptwitpiccomyzl', 'awww', 'thats', 'a', 'bummer', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', 'd'], words_clean=['switchfoot', 'httptwitpiccomyzl', 'awww', 'thats', 'bummer', 'shoulda', 'got', 'david', 'carr', 'third', 'day', 'd'], words_stemmed=['switchfoot', 'httptwitpiccomyzl', 'awww', 'that', 'bummer', 'shoulda', 'got', 'david', 'carr', 'third', 'day', 'd'], label=0.0, w2v=DenseVector([-0.0328, 0.0718, 0.0402, -0.1109, 0.0523, 0.0696, -0.0801, 0.0556, 0.0307, 0.0836, 0.0359, 0.0011, 0.0436

In [63]:
[train_w2v_df, test_w2v_df] = w2v_df.randomSplit([0.8, 0.2], seed = 123)

In [64]:
%%time
t1 = time.time()

lrModel_w2v = LogisticRegression(featuresCol="w2v").fit(train_w2v_df)
predictions_w2v = lrModel_w2v.transform(test_w2v_df)

t2 = time.time()
print('Time process Logistic Regression: {0:.2f} seconds'.format(t2-t1) )

23/08/02 23:56:20 WARN MemoryStore: Not enough space to cache rdd_401_0 in memory! (computed 65.0 MiB so far)
23/08/02 23:56:20 WARN BlockManager: Persisting block rdd_401_0 to disk instead.
23/08/02 23:56:27 WARN MemoryStore: Not enough space to cache rdd_401_0 in memory! (computed 272.9 MiB so far)
23/08/02 23:57:04 WARN MemoryStore: Not enough space to cache rdd_401_1 in memory! (computed 113.0 MiB so far)
23/08/02 23:57:04 WARN BlockManager: Persisting block rdd_401_1 to disk instead.
23/08/02 23:57:09 WARN MemoryStore: Not enough space to cache rdd_401_1 in memory! (computed 272.9 MiB so far)
23/08/02 23:57:10 WARN MemoryStore: Not enough space to cache rdd_401_0 in memory! (computed 272.9 MiB so far)
23/08/02 23:57:12 WARN MemoryStore: Not enough space to cache rdd_401_1 in memory! (computed 272.9 MiB so far)
23/08/02 23:57:13 WARN MemoryStore: Not enough space to cache rdd_401_0 in memory! (computed 272.9 MiB so far)
23/08/02 23:57:15 WARN MemoryStore: Not enough space to cache 

Time process Logistic Regression: 303.44 seconds
CPU times: user 117 ms, sys: 58.1 ms, total: 175 ms
Wall time: 5min 3s


In [65]:
evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
evaluator.evaluate(predictions_w2v)
--0.6510055694515825

0.6510055694515825

In [66]:
accuracy = predictions_w2v.filter(predictions_w2v.label == predictions_w2v.prediction).count() / float(test_w2v_df.count())
accuracy

0.7011376709702161

Twitter request

In [81]:
import twitter
consumer_key = "Z7dcDib4WaY3ObOztPNFWlRWF"
consumer_secret = "X8zMTelnyy8lA2wHXgDGEVDJ9AYlSm154ubwAzhWdtut57Cgiw"

access_token = "104798552-AyQLnMXTMPcZOd8k2tol8VizaNkGhg0Xq5FOdiBe"
access_token_secret = "YbUyPukqBjPSReWWAIklOYk3TcKzXWtJKt92h9AbAm53g"

api = twitter.Api(consumer_key=consumer_key,
                  consumer_secret=consumer_secret,
                  access_token_key=access_token,
                  access_token_secret=access_token_secret)

In [100]:
from requests_oauthlib import OAuth1Session
import os
import json
import traceback

# client_id = "ZVZVOVdQUGtpX0hPQlk1cUg0T1o6MTpjaQ"
# client_secrect = "29XZzBC1PlNVbyGjMRJ0gXdocYu6b8eq5_UVwmGrTjG7RTPemX"

consumer_key = "M7MtZySaFlSAXAVyPDgBi37Tu"
consumer_secret = "lwtlAyT4xFHONQZhCHzGUJhC2DQ9Sx19FWg6HU5UdHj1lqPH95"
# bearer_token = "AAAAAAAAAAAAAAAAAAAAAFzTpAEAAAAA3bICFfNe5M5dSAc5kvkxIwJTn6s%3DUdnBLNXc6rAVBGFScdMegc4ZOBUYhgH3lD993fxl119hKgkRQ8"

# User fields are adjustable, options include:
# created_at, description, entities, id, location, name,
# pinned_tweet_id, profile_image_url, protected,
# public_metrics, url, username, verified, and withheld
# fields = "created_at,description"
# params = {"user.fields": fields}

# Get request token
request_token_url = 'https://api.twitter.com/oauth/request_token'
oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)

try:
    fetch_response = oauth.fetch_request_token(request_token_url)
    print(fetch_response)
except ValueError:
    traceback.print_exc()
    print(
        "There may have been an issue with the consumer_key or consumer_secret you entered."
    )

# resource_owner_key = fetch_response.get("oauth_token")
# resource_owner_secret = fetch_response.get("oauth_token_secret")
# print("Got OAuth token: %s" % resource_owner_key)

# # # Get authorization
# base_authorization_url = "https://api.twitter.com/oauth/authorize"
# authorization_url = oauth.authorization_url(base_authorization_url)
# print("Please go here and authorize: %s" % authorization_url)
# verifier = input("Paste the PIN here: ")

# # Get the access token
# access_token_url = "https://api.twitter.com/oauth/access_token"
# oauth = OAuth1Session(
#     consumer_key,
#     client_secret=consumer_secret,
#     resource_owner_key=resource_owner_key,
#     resource_owner_secret=resource_owner_secret,
#     verifier=verifier,
# )
# oauth_tokens = oauth.fetch_access_token(access_token_url)

# access_token = "104798552-1LxJEtu9BtwGUNvXBimgjpwqgvpqU8937qa4qNkS"
# access_token_secret = "2qkQzPRkmfmiRQTqSDazIfVTl3wdNZj44OCdjVTjXkqhL"

# # # Make the request
# oauth = OAuth1Session(
#     client_key=consumer_key,
#     client_secret=consumer_secret,
#     resource_owner_key=access_token,
#     resource_owner_secret=access_token_secret,
# )

# response = oauth.get("https://api.twitter.com/2/users/me", params=params)

# if response.status_code != 200:
#     raise Exception(
#         "Request returned an error: {} {}".format(response.status_code, response.text)
#     )

# print("Response code: {}".format(response.status_code))

# json_response = response.json()

# print(json.dumps(json_response, indent=4, sort_keys=True))

There may have been an issue with the consumer_key or consumer_secret you entered.


Traceback (most recent call last):
  File "/tmp/ipykernel_48939/2192947765.py", line 25, in <cell line: 24>
    fetch_response = oauth.fetch_request_token(request_token_url)
  File "/var/root/miniconda3/lib/python3.8/site-packages/requests_oauthlib/oauth1_session.py", line 287, in fetch_request_token
    token = self._fetch_token(url, **request_kwargs)
  File "/var/root/miniconda3/lib/python3.8/site-packages/requests_oauthlib/oauth1_session.py", line 386, in _fetch_token
    self._populate_attributes(token)
  File "/var/root/miniconda3/lib/python3.8/site-packages/requests_oauthlib/oauth1_session.py", line 355, in _populate_attributes
    raise TokenMissing(
requests_oauthlib.oauth1_session.TokenMissing: Response does not contain a token: {}


In [75]:
main()

403


Exception: Request returned an error: 403 {
  "title": "Unsupported Authentication",
  "detail": "Authenticating with OAuth 2.0 Application-Only is forbidden for this endpoint.  Supported authentication types are [OAuth 1.0a User Context, OAuth 2.0 User Context].",
  "type": "https://api.twitter.com/2/problems/unsupported-authentication",
  "status": 403
}